In [1]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
sns.despine()
from scipy import interp
from statistics import mean
import matplotlib.pyplot as plt
from collections import OrderedDict

import sys
sys.path.append('../../scripts')
from helper_functions import *

### Load CS dataset generated from CATH FunFams

In [2]:
raw_feature_data = pd.read_csv('CS_feature_table.csv')
raw_feature_data.head(5)

,residue_string,A_pssm_ff,A_pssm_psiblast,A_wop_ff,A_wop_psiblast,C_pssm_ff,C_pssm_psiblast,C_wop_ff,C_wop_psiblast,D_pssm_ff,...,rsa_mainchain,rsa_nonpolar,rsa_polar,rsa_totside,sc5_gs,sc5_scons,scons,surface_residues_struc_neighbourhood,van_der_waals_vol_normalised,Unnamed: 170
0,1yonA02_168,-5.0,-3.0,0.0,0.0,-7.0,-5.0,0.0,0.0,5.0,...,140.1,100.4,103.8,89.5,-1,4,0.538,6,2.95,NaN
1,1yonA02_169,-5.0,-4.0,0.0,0.0,-5.0,-4.0,0.0,0.0,-7.0,...,7.2,80.3,7.5,81.0,-1,5,0.737,6,4.00,NaN
2,1yonA02_170,-1.0,1.0,4.0,13.0,-4.0,-3.0,0.0,0.0,-1.0,...,1.1,66.9,45.2,61.8,-1,2,0.369,9,6.13,NaN
3,1yonA02_171,1.0,3.0,15.0,36.0,-3.0,-2.0,0.0,0.0,-1.0,...,17.3,34.9,15.6,34.5,-1,2,0.371,8,1.00,NaN
4,1yonA02_172,3.0,4.0,27.0,49.0,-3.0,-2.0,0.0,0.0,-1.0,...,7.9,77.6,59.4,81.9,-1,3,0.402,8,3.78,NaN


In [3]:
raw_feature_data.columns.tolist()

['residue_string',
 'A_pssm_ff',
 'A_pssm_psiblast',
 'A_wop_ff',
 'A_wop_psiblast',
 'C_pssm_ff',
 'C_pssm_psiblast',
 'C_wop_ff',
 'C_wop_psiblast',
 'D_pssm_ff',
 'D_pssm_psiblast',
 'D_wop_ff',
 'D_wop_psiblast',
 'E_pssm_ff',
 'E_pssm_psiblast',
 'E_wop_ff',
 'E_wop_psiblast',
 'F_pssm_ff',
 'F_pssm_psiblast',
 'F_wop_ff',
 'F_wop_psiblast',
 'G_pssm_ff',
 'G_pssm_psiblast',
 'G_wop_ff',
 'G_wop_psiblast',
 'H_pssm_ff',
 'H_pssm_psiblast',
 'H_wop_ff',
 'H_wop_psiblast',
 'I_pssm_ff',
 'I_pssm_psiblast',
 'I_wop_ff',
 'I_wop_psiblast',
 'K_pssm_ff',
 'K_pssm_psiblast',
 'K_wop_ff',
 'K_wop_psiblast',
 'L_pssm_ff',
 'L_pssm_psiblast',
 'L_wop_ff',
 'L_wop_psiblast',
 'M_pssm_ff',
 'M_pssm_psiblast',
 'M_wop_ff',
 'M_wop_psiblast',
 'N_pssm_ff',
 'N_pssm_psiblast',
 'N_wop_ff',
 'N_wop_psiblast',
 'P_pssm_ff',
 'P_pssm_psiblast',
 'P_wop_ff',
 'P_wop_psiblast',
 'Q_pssm_ff',
 'Q_pssm_psiblast',
 'Q_wop_ff',
 'Q_wop_psiblast',
 'R_pssm_ff',
 'R_pssm_psiblast',
 'R_wop_ff',
 'R_wop_psiblast',
 'SF',
 'S_pssm_ff',
 'S_pssm_psiblast',
 'S_wop_ff',
 'S_wop_psiblast',
 'T_pssm_ff',
 'T_pssm_psiblast',
 'T_wop_ff',
 'T_wop_psiblast',
 'V_pssm_ff',
 'V_pssm_psiblast',
 'V_wop_ff',
 'V_wop_psiblast',
 'W_pssm_ff',
 'W_pssm_psiblast',
 'W_wop_ff',
 'W_wop_psiblast',
 'Y_pssm_ff',
 'Y_pssm_psiblast',
 'Y_wop_ff',
 'Y_wop_psiblast',
 'alpha',
 'annotation_3DID_INTERCHAIN',
 'annotation_3DID_INTRACHAIN',
 'annotation_BIOLIP',
 'annotation_BIOLIP_ligand',
 'annotation_IBIS_LIG',
 'annotation_IBIS_PPI_INTERCHAIN',
 'annotation_IBIS_PPI_INTRACHAIN',
 'annotation_MCSA',
 'annotation_MCSA_role',
 'annotation_MCSA_roletype',
 'annotation_PROTINDB',
 'avg_charged',
 'avg_cx',
 'avg_dpx',
 'avg_electric_effect',
 'avg_flexibility',
 'avg_hydropathicity',
 'avg_hydrophobicity',
 'avg_polarity',
 'avg_scons',
 'avg_surface_residues',
 'avg_surrounding_hydrophobicity',
 'betweenness',
 'bulkiness',
 'charge',
 'cleft_depth',
 'cleft_num',
 'closeness',
 'conserved_hotspot_struc_neighbourhood',
 'conserved_surface_hotspot_struc_neighbourhood',
 'degree',
 'dist_to_hotspot',
 'dist_to_surface',
 'dom_group',
 'domain_length',
 'domain_num_in_chain',
 'dops',
 'dssp_type',
 'entwop_score_ff',
 'entwop_score_psiblast',
 'foldx_alascan',
 'free_energy_solution',
 'gapless_match_to_pseudocounts_ff',
 'gapless_match_to_pseudocounts_psiblast',
 'highly_conserved_struc_neighbourhood',
 'highly_conserved_surface_struc_neighbourhood',
 'hotspot_struc_neighbourhood',
 'hydration_potential',
 'hydropathicity',
 'hydropathy_index',
 'hydrophobicity',
 'hydrophobicity_psaia',
 'ibis_lig_freq',
 'ibis_ppi_freq',
 'info_per_pos_ff',
 'info_per_pos_psiblast',
 'kappa',
 'localised_electrical_effect',
 'max_cx',
 'max_dpx',
 'min_cx',
 'min_dist_to_cleft_1',
 'min_dist_to_cleft_2',
 'min_dist_to_cleft_3',
 'min_dpx',
 'mutability',
 'nhBonds_ptr',
 'oBonds_ptr',
 'phi',
 'pocket_conserved_struc_neighbourhood',
 'pocket_struc_neighbourhood',
 'pocket_surface_conserved_struc_neighbourhood',
 'polarity',
 'psi',
 'resTco',
 'res_bfactor_n',
 'residue_aa',
 'rsa_allatoms',
 'rsa_mainchain',
 'rsa_nonpolar',
 'rsa_polar',
 'rsa_totside',
 'sc5_gs',
 'sc5_scons',
 'scons',
 'surface_residues_struc_neighbourhood',
 'van_der_waals_vol_normalised',
 'Unnamed: 170']

In [4]:
# feature engineering
raw_feature_data = raw_feature_data.drop(['Unnamed: 170'], axis =1)
raw_feature_data['domain'], raw_feature_data['domain_residue'] = raw_feature_data['residue_string'].str.split('_', 1).str
raw_feature_data['dssp_type'] = raw_feature_data['dssp_type'].fillna("NO_PRED")
raw_feature_data['surface_residue_rsa'] = (raw_feature_data['rsa_allatoms'] >= 25).astype(int)
raw_feature_data['highly_conserved'] = (raw_feature_data['scons'] >= 0.7).astype(int)
raw_feature_data['cleft_residue'] = (raw_feature_data['cleft_num'] > 0).astype(int)
raw_feature_data['hydrophobic_aa'] = (raw_feature_data['hydrophobicity'] >= 0.48).astype(int)
raw_feature_data['polar_aa'] = (raw_feature_data['polarity'] >= 10).astype(int)
raw_feature_data['res_bfactor_n'] = raw_feature_data['res_bfactor_n'].astype(float)
raw_feature_data['entwop_score_ff'] = raw_feature_data['entwop_score_ff'].astype(float)
raw_feature_data['entwop_score_psiblast'] = raw_feature_data['entwop_score_psiblast'].astype(float)
mindist = raw_feature_data[['min_dist_to_cleft_1','min_dist_to_cleft_2','min_dist_to_cleft_3']].min(axis=1)
raw_feature_data = raw_feature_data.assign(min_dist_to_cleft123=mindist)
raw_feature_data = pd.get_dummies(raw_feature_data, columns=['residue_aa', 'dssp_type'])
# Remove any duplicate samples. 
raw_feature_data = raw_feature_data.drop_duplicates()
# Remove NA rows
raw_feature_data = raw_feature_data.dropna()
# Count no. of domains in whole dataset
raw_feature_data.groupby(['domain']).size().shape[0]

691

### Generate CS benchmark and validation datasets

In [5]:
csmetapred_df = pd.read_csv('CSmetapred_predictions.txt', names = ['domain_csmetapred','res', 'dom_res', 'rank','csmetapred', 'exia2', 'crpred', 'discern'], sep='\t')
csmetapred_df.head(5)
# Get domains in whole dataset common with benchmark dataset
raw_feature_data_csmetapred =raw_feature_data.merge(csmetapred_df, left_on='residue_string', right_on='dom_res', how='inner')

In [6]:
#### Get CS Validation dataset
file='T124_benchmark_dataset.tab'
validation_domlist = get_validation_list(file)
validation_set = raw_feature_data[raw_feature_data['domain'].isin(validation_domlist)]
validation_set_csmetapred = validation_set.set_index('residue_string').join(csmetapred_df.set_index('dom_res'), how = 'inner', lsuffix='_ff', rsuffix='_csmetapred')
validation_set_csmetapred.head(5)

,A_pssm_ff,A_pssm_psiblast,A_wop_ff,A_wop_psiblast,C_pssm_ff,C_pssm_psiblast,C_wop_ff,C_wop_psiblast,D_pssm_ff,D_pssm_psiblast,...,dssp_type_H,dssp_type_NO_PRED,dssp_type_T,domain_csmetapred,res,rank,csmetapred,exia2,crpred,discern
1czfA00_100,-1.0,-1.0,3.0,2.0,-3.0,-3.0,0.0,0.0,2.0,0.0,...,0,0,0,1czfA00,100,249,0.196086,0.104497,0.033381,0.338205
1czfA00_101,-1.0,0.0,3.0,6.0,-6.0,0.0,0.0,2.0,-5.0,-4.0,...,0,0,0,1czfA00,101,101,0.368455,0.285269,0.047977,0.672939
1czfA00_102,1.0,3.0,13.0,35.0,-2.0,-2.0,1.0,0.0,-1.0,0.0,...,0,1,0,1czfA00,102,251,0.192415,0.285269,0.035455,0.456369
1czfA00_103,0.0,0.0,6.0,6.0,-1.0,-4.0,1.0,0.0,0.0,1.0,...,0,0,1,1czfA00,103,313,0.052956,0.023789,0.100103,0.245220
1czfA00_104,-2.0,-2.0,2.0,1.0,-2.0,-3.0,1.0,0.0,-1.0,2.0,...,0,0,1,1czfA00,104,192,0.256773,0.285269,0.180394,0.614327


In [7]:
### Generate Training data
training_data=raw_feature_data[~raw_feature_data['domain'].isin(validation_domlist)]
training_data.groupby(['domain']).size().shape[0]
# Remove identical domains in dataset if present
redundant_domains=['1n2cF00', '1g64B00', '1xs1C00', '1c4tC00', '1ax4B02']
training_data_nr=training_data[~training_data['domain'].isin(redundant_domains)]

In [8]:
# Hold out dataset
validation_set_csmetapred_nr=validation_set_csmetapred[~validation_set_csmetapred['domain_csmetapred'].isin(redundant_domains)]
validation_set_csmetapred_nr=validation_set_csmetapred_nr.reset_index()
validation_set_csmetapred_nr.rename(columns={'index':'residue_string'}, inplace=True)

In [9]:
# Training dataset
SITE_data = training_data_nr[(training_data_nr.annotation_MCSA == 1) ]
site_doms = SITE_data.domain.unique()
NOSITE_data = training_data_nr[(training_data_nr.annotation_MCSA == 0 ) & training_data_nr.domain.isin(site_doms)] 
NOSITE_data['index']=NOSITE_data['residue_string']
SITE_data['index']=SITE_data['residue_string']

/Users/sayonidas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/sayonidas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
def preprocess_site_nonsite_df(site_data, nonsite_data, ratio):
    
    # Get the list of domains that have site annotations
    positive_sample_num = site_data.shape[0]
    print('#Postive samples:',positive_sample_num)
    
    # make a subset dataframe containing only NOSITE (negative) residues
    negative_sample_num = nonsite_data.shape[0]
    print('#Negative samples:',negative_sample_num)
    
    print ('Use these samples for training the model:')
    use_sample_num = positive_sample_num * ratio
        
    if (use_sample_num > negative_sample_num):
        use_sample_num = negative_sample_num
    
    total_samples = use_sample_num + positive_sample_num
    
    print ('- Used negative samples:',use_sample_num)
    print ('- Total samples:',total_samples)
    
    dom_groups_df = nonsite_data.groupby(['dom_group']).size()
    dom_group_num = dom_groups_df.shape[0]
    print ('- No. of groups of samples:',dom_group_num)
    
    sample_size = round(use_sample_num/dom_group_num)

    print ('- Min. sample size in no_sites:', sample_size)
    
    nonsite_data = nonsite_data.groupby(['dom_group']).filter(lambda x: len(x) > sample_size)
    nonsite_data_randomsubset=nonsite_data.groupby('dom_group').apply(lambda x: x.sample(n=sample_size, random_state=10)).reset_index(drop=True)
    
    # COMBINE selected csa and non-csa data for the desired dataset ratio
    frames = [site_data, nonsite_data_randomsubset]
    concatenated_feature_data = pd.concat(frames)
    dataset_sample_num = concatenated_feature_data.shape[0]
    feature_data_ML = concatenated_feature_data.set_index('index').sample(n=dataset_sample_num, random_state=10)
    feature_data_ML.index.name = None
    
    return(feature_data_ML)

In [11]:
training_data_ML = preprocess_site_nonsite_df(SITE_data, NOSITE_data, 6)
training_data_ML = training_data_ML.drop_duplicates()
len(training_data_ML.domain.unique())

#Postive samples: 1939
#Negative samples: 120723
Use these samples for training the model:
- Used negative samples: 11634
- Total samples: 13573
- No. of groups of samples: 565
- Min. sample size in no_sites: 21


565

In [12]:
SITE_data_val = validation_set_csmetapred_nr[(validation_set_csmetapred_nr.annotation_MCSA == 1) ]
site_doms_val = SITE_data_val.dom_group.unique()
NOSITE_data_val = validation_set_csmetapred_nr[(validation_set_csmetapred_nr.annotation_MCSA == 0 ) & (validation_set_csmetapred_nr.dom_group.isin(site_doms_val)) ]
NOSITE_data_val['index']=NOSITE_data_val['residue_string']
SITE_data_val['index']=SITE_data_val['residue_string']
validation_data_ML = preprocess_site_nonsite_df(SITE_data_val, NOSITE_data_val, 6)
validation_data_ML =validation_data_ML.drop_duplicates()
validation_data_ML.groupby('domain_csmetapred').size().shape[0]

/Users/sayonidas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/sayonidas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#Postive samples: 329
#Negative samples: 21095
Use these samples for training the model:
- Used negative samples: 1974
- Total samples: 2303
- No. of groups of samples: 102
- Min. sample size in no_sites: 19


102

In [13]:
training_data_ML = training_data_ML.sort_values(['domain'])
validation_data_ML = validation_data_ML.sort_values(['domain_csmetapred'])

training_data_ML.to_csv('CS_training_dataset.csv', index=False)
validation_data_ML.to_csv('CS_validation_dataset.csv', index=False)